# How To: Neural networks in the observables

This is the case where a neural network appears in an observable formula.  The SBML model for the Lotka-Voltera system would be:

$$\frac{\mathrm{d} \text{prey}}{\mathrm{d} t} = \alpha \cdot \text{prey} - \beta \cdot \text{prey} \cdot \text{predator}$$

$$\frac{\mathrm{d} \text{predator}}{\mathrm{d} t} = \gamma \cdot \text{prey} \cdot \text{predator} - \delta \cdot \text{predator}$$

No species in these expressions are substituted. Instead it is all accomplished in the PEtab files.

## Loading the PEtab problem

In [ ]:
from amici.petab import import_petab_problem
from amici.jax import (
    JAXProblem,
    run_simulations,
)
from petab.v2 import Problem

# Create the PEtab problem
petab_problem = Problem.from_yaml("problem.yaml")

# Create AMICI model for the petab problem
jax_model = import_petab_problem(
    petab_problem,
    model_output_dir="model",
    compile_=True,
    jax=True
)

# Create the JAXProblem - wrapper for the AMICI model
jax_problem = JAXProblem(jax_model, petab_problem)

In [3]:
petab_problem.observable_df

,observableFormula,noiseFormula,observableTransformation,noiseDistribution
observableId,,,,
prey_o,net1_output1,0.05,lin,normal
predator_o,predator,0.05,lin,normal


We can see here that the formula for the ``prey`` observable is defined by a PEtab identifier indicating a network output. The model definition of that PEtab id can be found in the mapping table.  It is given by the first output of the neural network.

In [5]:
petab_problem.mapping_df

,modelEntityId
petabEntityId,
net1_input1,net1.inputs[0][0]
net1_input2,net1.inputs[0][1]
net1_output1,net1.outputs[0][0]
net1_ps,net1.parameters


Wherever the problem formulation refers to the prey observable (``prey_o``) the neural network will be evaluated and its output used in place of that observable value, for instance, in the measurements table.

In [7]:
petab_problem.measurement_df

,observableId,simulationConditionId,measurement,time
0,prey_o,cond1,0.173017,1.0
1,prey_o,cond1,0.489177,2.0
2,prey_o,cond1,1.643996,3.0
3,prey_o,cond1,5.451963,4.0
4,prey_o,cond1,2.977522,5.0
5,prey_o,cond1,0.181663,6.0
6,prey_o,cond1,0.348112,7.0
7,prey_o,cond1,0.937919,8.0
8,prey_o,cond1,3.113240,9.0
9,prey_o,cond1,8.863933,10.0


**And that is pretty much it for this one?**

In the observable table I could make the formula more complex - e.g. petab can parse equation expressions 
sum(net1_output1) + 2